# Preprocessing

In [ ]:
# import os
# from random import randint, uniform
# import re
# import numpy as np
# import wave
# import contextlib

## Data Preparation for Recording

In [19]:
MAX_SENTENCE_LEN = 20
SOURCE_PATH = 'yor_trans.txt'
DEST_TEMP_PATH = 'yor_split.txt'
DEST_TEMP_CLEAN_PATH = 'yor_clean_split.txt'
NUM_FOLDER_SPLIT = 50
FOLDER_PATH="split_text"
PARTIAL_NAME= "yor_split"

In [20]:
from yor_processor import split_file, split_file_into_folders

In [21]:
status = split_file(SOURCE_PATH, DEST_TEMP_PATH, MAX_SENTENCE_LEN)
# clean_file_status = split_file(SOURCE_PATH, DEST_TEMP_CLEAN_PATH, MAX_SENTENCE_LEN, end_of_file="\n")
if status == 'done':
    split_file_into_folders(FOLDER_PATH, DEST_TEMP_PATH, PARTIAL_NAME, NUM_FOLDER_SPLIT)

## Calculate Total Recording Lengths

In [22]:
DIR_OF_REC = "./recordings/"
FILE_FORMAT = ".wav"

In [23]:
from yor_processor import calculate_recording_len

total_len, good_files, corrupted_files = calculate_recording_len(DIR_OF_REC, FILE_FORMAT)
print("total len of recording is ", round(total_len, 2), "s",", ",total_len/(60*60),"h")

total len of recording is  11030.87 s ,  3.0641296354166636 h


In [24]:
print(good_files)
print(corrupted_files)

1079
30


# Organizing data

### Spliting records to train, val and test sets

In [25]:
!mkdir data
!mkdir data/records

In [26]:
from yor_processor import extract_non_corrupted_files

wav_files, all_linkers = extract_non_corrupted_files(DIR_OF_REC)

In [27]:
all_linkers.keys()

dict_keys(['200625-230031_yor_874_elicit', '200630-123859_yor_874_elicit', '200630-112655_yor_874_elicit', '200629-222626_yor_874_elicit', '200629-234938_yor_874_elicit', '200630-092358_yor_874_elicit', '200630-110532_yor_874_elicit', '200625-215447_yor_874_elicit', '200630-101743_yor_874_elicit', '200629-230040_yor_874_elicit', '200625-221631_yor_874_elicit', '200630-003859_yor_874_elicit', '200626-001537_yor_874_elicit', '200625-223759_yor_874_elicit', '200630-095103_yor_874_elicit', '200629-232814_yor_874_elicit', '200625-232021_yor_874_elicit', '200630-115031_yor_874_elicit', '200629-205402_yor_874_elicit', '200625-235253_yor_874_elicit', '200630-121116_yor_874_elicit', '200630-001255_yor_874_elicit'])

In [28]:
# copy wav files
to_copy = " ".join(wav_files)
!cp -t data/records/ {to_copy}

In [29]:
new_linkers = dict()
for section, linker in all_linkers.items():
    for i,link in enumerate(linker):
        text_file_name = link.split(" ")[0].split("/")[-1]
        line = link.split(";")[0].split("(")[1].split(")")[0].strip()
        wav = link.split(";")[1].strip().split("/")[-1]

        linker[i] = wav+":"+line
        new_linkers[text_file_name] = linker

In [30]:
!mkdir ./data/records/train
!mkdir ./data/records/test
!mkdir ./data/records/val
!mkdir ./data/records/extra

In [31]:
from yor_processor import split_train_val_test

# reduce to 2hrs data by splitting into 3 with 3hrs of data
# split 1hr into train and val
to_copy_train, to_copy_valid, to_copy_test, to_copy_extra = split_train_val_test(wav_files, 
                                                                                 num_splits=3, # reduce to 2hrs data since I have 3hrs of data
                                                                                 val_split=0.2)

In [32]:
!mv -t data/records/train/ {to_copy_train}
!mv -t data/records/val/ {to_copy_valid}
!mv -t data/records/test/ {to_copy_test}
!mv -t data/records/extra/ {to_copy_extra}

### Make chars.txt file

In [33]:
from yor_processor import create_char_set

chars_list, text_data  = create_char_set(new_linkers, path="./split_text/", exclude="[\n\.,''-''̀''́'''!-]")

In [34]:
print(chars_list)

{' ': 1, 'ε': 0, 'à': 2, 'é': 3, 'l': 4, 'i': 5, 'r': 6, 'ṣ': 7, 'ò': 8, 'p': 9, 'v': 10, 'ú': 11, 'n': 12, 'b': 13, 'o': 14, 'y': 15, 'ọ': 16, 'd': 17, 'g': 18, 'e': 19, 't': 20, 'k': 21, 'ì': 22, 'j': 23, 'á': 24, 'è': 25, 'ù': 26, 'ẹ': 27, 'a': 28, 'í': 29, 's': 30, 'h': 31, 'w': 32, 'f': 33, 'ó': 34, 'm': 35, 'u': 36, 'c': 37, '–': 38, 'ń': 39, 'ǹ': 40, 'z': 41, 'ḿ': 42, 'ί': 43, 'ὸ': 44}


### Create data in format for training

In [35]:
from yor_processor import create_data_format

create_data_format(text_data, chars_list)

files created for training


In [36]:
len(chars_list)

45